In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/studentcnn/pytorch/default/1/model (4).pth


In [2]:
!pip install detectors -q


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [4]:
import torch,timm,detectors
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import KLDivLoss
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = timm.create_model("resnet34_cifar10", pretrained=True).to(device)


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


  0%|          | 0.00/170M [00:00<?, ?B/s]

  0%|          | 65.5k/170M [00:00<08:10, 347kB/s]

  0%|          | 164k/170M [00:00<04:39, 610kB/s] 

...

 74%|███████▍  | 60.2M/81.3M [00:01<00:00, 56.3MB/s]

 83%|████████▎ | 67.2M/81.3M [00:01<00:00, 60.9MB/s]

 91%|█████████ | 73.6M/81.3M [00:01<00:00, 53.8MB/s]

 98%|█████████▊| 79.9M/81.3M [00:01<00:00, 56.6MB/s]

100%|██████████| 81.3M/81.3M [00:01<00:00, 58.3MB/s]

In [5]:
# Define the StudentCNN model
class StudentCNN(nn.Module):
    def __init__(self):
        super(StudentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Adjusted for CIFAR-10
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [6]:
def train_with_kl_and_ce(student_model, teacher_model, loader, optimizer, epochs=50, temperature=30, alpha=0.7):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    ce_loss = nn.CrossEntropyLoss()
    student_model.train()

    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher_model(images)
                teacher_probs = F.softmax(teacher_logits / temperature, dim=1)

            optimizer.zero_grad()
            student_logits = student_model(images)

            # KL Divergence Loss
            student_log_probs = F.log_softmax(student_logits / temperature, dim=1)
            loss_kl = kl_loss(student_log_probs, teacher_probs) * (temperature ** 2)

            # Cross-Entropy Loss
            loss_ce = ce_loss(student_logits, labels)

            # Combined Loss
            loss = alpha * loss_kl + (1 - alpha) * loss_ce

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Total Loss: {total_loss:.4f}")


In [7]:
student_model_kl = StudentCNN()

In [8]:


student_model_kl.to(device)
optimizer_kl = optim.Adam(student_model_kl.parameters(), lr=0.001)

train_with_kl_and_ce(student_model_kl, teacher_model, train_loader, optimizer_kl)

Epoch 1, Total Loss: 2437.8152


Epoch 2, Total Loss: 1807.4812


Epoch 3, Total Loss: 1513.0496


Epoch 4, Total Loss: 1291.7191


Epoch 5, Total Loss: 1091.4017


...

Epoch 46, Total Loss: 185.4066


Epoch 47, Total Loss: 180.3852


Epoch 48, Total Loss: 181.1360


Epoch 49, Total Loss: 184.2387


Epoch 50, Total Loss: 181.1936


In [9]:
torch.save(student_model_kl, '/kaggle/working/model.pth')

In [10]:
correct = 0
total = 0
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
with torch.no_grad():
    for images, labels in test_loader:
        outputs = student_model_kl(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 74.10%
